In [1]:
%cd ..

/home/dina/Desktop/grad project/My code/Abnormal-Event-Detection-


In [2]:
import numpy as np
from tensorflow.keras.models import load_model
import os
import h5py
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from evaluate import plot_regularity_score, plot_reconstruction_error,calc_auc_overall
from PIL import Image
import re
from Code.testing import t_predict_video,video_to_clips
from scipy import signal
from matplotlib import pyplot as plt

In [3]:
%ls

Code/       default-b894b6f8-6e27-4aed-94b4-bcfd6f4f341b.ipynb  README.md
config.yml  images/
Data/       LICENSE


In [4]:
#%cd ..
dataset = 'UCSDped1'
epoch = 50
val_loss = -0.117306
t = 10 #time length of each clip

#fetching paths to test_data, job_folder and trained model
test_dir = os.path.join('Data/{0}/testing_numpy'.format(dataset))
model_folder = os.path.join('./Data/{0}'.format(dataset))
model_filename = 'model_snapshot_e{:03d}_{:.6f}.h5'.format(epoch, val_loss)

#load model
temporal_model = load_model(os.path.join(model_folder, model_filename))

In [5]:
#loop on all videos in the test data
test_video = 'Test001_corr.mp4'
test_video = test_video.replace('.mp4','.npy')
filepath = os.path.join(test_dir, test_video)
X_test = np.load(filepath)

In [6]:
X_test.shape

(200, 227, 227)

In [7]:
sequences,sz = video_to_clips(X_test,t)

In [8]:
sequences.shape

(191, 10, 227, 227, 1)

In [9]:
predicted = temporal_model.predict(sequences)

In [10]:
predicted.shape

(191, 10, 227, 227, 1)

In [95]:
def plot(img, output, diff, H, threshold, counter):
    
    fig, (ax0, ax1, ax2,ax3) = plt.subplots(ncols=4, figsize=(10, 5))
    ax0.set_axis_off()
    ax1.set_axis_off()
    ax2.set_axis_off()
    
    ax0.set_title('input image')
    ax1.set_title('reconstructed image')
    ax2.set_title('diff ')
    ax3.set_title('anomalies')
    
    ax0.imshow(img, cmap=plt.cm.gray, interpolation='nearest') 
    ax1.imshow(output, cmap=plt.cm.gray, interpolation='nearest')   
    ax2.imshow(diff, cmap=plt.cm.viridis, vmin=0, vmax=255, interpolation='nearest')  
    ax3.imshow(img, cmap=plt.cm.gray, interpolation='nearest')
    
    x,y = np.where(H > threshold)
    ax3.scatter(y,x,color='red',s=0.1) 

    plt.axis('off')
    plt.close('all')  
    fig.savefig('images/' +str(counter) + '.png')

In [96]:
mean = np.load(os.path.join('Data/{0}/mean_frame_224.npy'.format(dataset)))
mean = np.expand_dims(mean,axis=-1)

In [101]:
try:
    os.mkdir("images")
except:
    pass
counter = 0
threshold = 6*256
for i,vol in enumerate(predicted):
    print('vol ===> ', i)
    img = np.expand_dims(X_test[i],axis=-1) + (mean/8)
    #print(mean.shape)
    counter = counter + 1
    output = vol[0] + mean
    #print(output.shape)
    output = np.asarray(output)*256
    img = np.asarray(img)*256
    diff = np.abs(output-img)**2
    tmp = diff[:,:,0]
    H = signal.convolve2d(tmp, np.ones((4,4)), mode='same')
    plot(img[:,:,0], output[:,:,0], diff[:,:,0], H, threshold, counter)


vol ===>  0
vol ===>  1
vol ===>  2
vol ===>  3
vol ===>  4
vol ===>  5
vol ===>  6
vol ===>  7
vol ===>  8
vol ===>  9
vol ===>  10
vol ===>  11
vol ===>  12
vol ===>  13
vol ===>  14
vol ===>  15
vol ===>  16
vol ===>  17
vol ===>  18
vol ===>  19
vol ===>  20
vol ===>  21
vol ===>  22
vol ===>  23
vol ===>  24
vol ===>  25
vol ===>  26
vol ===>  27
vol ===>  28
vol ===>  29
vol ===>  30
vol ===>  31
vol ===>  32
vol ===>  33
vol ===>  34
vol ===>  35
vol ===>  36
vol ===>  37
vol ===>  38
vol ===>  39
vol ===>  40
vol ===>  41
vol ===>  42
vol ===>  43
vol ===>  44
vol ===>  45
vol ===>  46
vol ===>  47
vol ===>  48
vol ===>  49
vol ===>  50
vol ===>  51
vol ===>  52
vol ===>  53
vol ===>  54
vol ===>  55
vol ===>  56
vol ===>  57
vol ===>  58
vol ===>  59
vol ===>  60
vol ===>  61
vol ===>  62
vol ===>  63
vol ===>  64
vol ===>  65
vol ===>  66
vol ===>  67
vol ===>  68
vol ===>  69
vol ===>  70
vol ===>  71
vol ===>  72
vol ===>  73
vol ===>  74
vol ===>  75
vol ===>  76
vol ===> 

In [102]:
import os
import cv2
import numpy as np

fps = 25.0
pathOut = 'images/video.mp4'
pathOut2 = 'images/'+'video_corr'+'.mp4'
if pathOut:
    frame_array = []
    print(len(sorted(os.listdir('images'))))
    for frame_file in range(1,len(sorted(os.listdir('images')))-1):
        img = cv2.imread(os.path.join('images/',str(frame_file)+'.png'))
        print(os.path.join('images/',str(frame_file)+'.png'))
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)
    #frame_array = sort_nicely(frame_array)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()
    os.system('ffmpeg -i {0} -vcodec libx264 {1}'.format(pathOut,pathOut2))
    #os.remove(pathOut)
    

191
images/1.png
images/2.png
images/3.png
images/4.png
images/5.png
images/6.png
images/7.png
images/8.png
images/9.png
images/10.png
images/11.png
images/12.png
images/13.png
images/14.png
images/15.png
images/16.png
images/17.png
images/18.png
images/19.png
images/20.png
images/21.png
images/22.png
images/23.png
images/24.png
images/25.png
images/26.png
images/27.png
images/28.png
images/29.png
images/30.png
images/31.png
images/32.png
images/33.png
images/34.png
images/35.png
images/36.png
images/37.png
images/38.png
images/39.png
images/40.png
images/41.png
images/42.png
images/43.png
images/44.png
images/45.png
images/46.png
images/47.png
images/48.png
images/49.png
images/50.png
images/51.png
images/52.png
images/53.png
images/54.png
images/55.png
images/56.png
images/57.png
images/58.png
images/59.png
images/60.png
images/61.png
images/62.png
images/63.png
images/64.png
images/65.png
images/66.png
images/67.png
images/68.png
images/69.png
images/70.png
images/71.png
images/72.p